In [39]:
import pandas as pd
import importlib

In [86]:
import basic_example2 as be
importlib.reload(be)

<module 'basic_example2' from '/home/julia/school/bakalarka/hospital_sim/basic_example2.py'>

In [87]:
nurses = []
for i in range(2):
    nurses.append(be.Nurse(i))
patients = []
for i in range(3):
    patients.append(be.Patient(i, i % len(nurses)))


In [130]:
requests = be.generate_requests(len(patients))
sim = be.Simulator(requests, nurses, patients)
df = pd.DataFrame(sim.simulate())

[(55.35660626238603, 2), (62.946804119286256, 0), (64.74100340897758, 2), (70.14272257559963, 1), (71.38833074409456, 2), (72.41967173501043, 1), (124.49685815331866, 2), (134.54782654154056, 1), (182.1138087766049, 2), (183.90624773717613, 1), (195.26068774804193, 0), (219.6573639757521, 2), (224.14046757763975, 0), (224.47745618686085, 1), (232.40505006817534, 0), (291.08694698552847, 0), (291.6739296866072, 0), (314.3760073320478, 1), (314.39888987634475, 0), (332.8978079356482, 0), (351.2232250111561, 0), (365.21397049942414, 1), (385.0461358783523, 1), (490.46530440811244, 0), (519.9751010120759, 1), (548.116338892045, 2), (561.3482013406147, 1), (579.7042064921906, 1), (584.8882415295295, 1), (595.1998990391451, 0), (602.5256413209252, 2), (608.1639370017492, 0), (631.4394404443234, 0), (643.7554563945608, 1), (644.625030558521, 1), (649.2302466699371, 1), (658.1125994485897, 0), (660.1789614385472, 0), (677.7956026611932, 1), (691.408002855256, 2), (692.0344428796747, 0), (710.3

In [113]:
requests = [(1, 0), (25, 2), (26, 1), (50, 0)] #time, patient id
sim = be.Simulator(requests, nurses, patients)
df = pd.DataFrame(sim.simulate())

1 request from patient 0
1 nurse 0 chosen for patient 0
2 nurse 0 departing to patient 0
12 nurse 0 arrived at patient 0
25 request from patient 2
25 nurse 1 chosen for patient 2 because nurse 0 is busy
26 request from patient 1
26 all nurses busy, request pushed back to 42
26 nurse 1 departing to patient 2
32 nurse 0 dealt with request from patient 0
36 nurse 1 arrived at patient 2
42 request from patient 1
42 all nurses busy, request pushed back to 42
42 nurse 0 returned from patient 0
42 request from patient 1
42 nurse 0 chosen for patient 1 because nurse 1 is busy
43 nurse 0 departing to patient 1
50 request from patient 0
50 all nurses busy, request pushed back to 66
53 nurse 0 arrived at patient 1
56 nurse 1 dealt with request from patient 2
66 request from patient 0
66 all nurses busy, request pushed back to 66
66 nurse 1 returned from patient 2
66 request from patient 0
66 nurse 1 chosen for patient 0 because nurse 0 is busy
67 nurse 1 departing to patient 0
73 nurse 0 dealt wi

In [131]:
df

,time,type,request_id,patient_id,requested_nurse_id,chosen_nurse_id,pushed_back,nurse_id,nurse_phase,pushed_back_time
0,55.356606,request,0,2,0.0,0.0,False,NaN,NaN,NaN
1,56.356606,nurse_phase,0,2,NaN,NaN,NaN,0.0,departing,NaN
2,62.946804,request,1,0,0.0,1.0,False,NaN,NaN,NaN
3,63.946804,nurse_phase,1,0,NaN,NaN,NaN,1.0,departing,NaN
4,64.741003,request,2,2,0.0,NaN,True,NaN,NaN,96.356606
...,...,...,...,...,...,...,...,...,...,...
1706,3636.685618,nurse_phase,162,0,NaN,NaN,NaN,1.0,dealt with request,NaN
1707,3644.213059,nurse_phase,163,0,NaN,NaN,NaN,0.0,arrived,NaN
1708,3646.685618,nurse_phase,162,0,NaN,NaN,NaN,1.0,returned,NaN
1709,3664.213059,nurse_phase,163,0,NaN,NaN,NaN,0.0,dealt with request,NaN


In [50]:
#kolikrát sestra vyřizovala pacienta
def patients_per_nurse(nurse_id):
    nurse_df =  df[df["chosen_nurse_id"] == nurse_id]
    display(nurse_df["patient_id"].value_counts())

In [78]:
patients_per_nurse(0)

patient_id
0    1
1    1
Name: count, dtype: int64

In [77]:
#kolik toho sestra nachodila
def nurse_distance_walked(nurse_id):
    nurse_df = df[df["nurse_id"] == nurse_id]
    times = nurse_df["time"].to_list()
    assert(len(times) % 4 == 0)
    request_num = (len(times) / 4)
    walk_time_there = times[1] - times[0]
    walk_time_back = times[3] - times[2]
    walk_time = request_num * (walk_time_there + walk_time_back)
    # display(nurse_df)
    return walk_time

print(nurse_distance_walked(1))

20.0


In [143]:
#jak dlouho pacient čekal
def total_wait_time(patient_id):
    patient_df = df[df["patient_id"] == patient_id]
    display(patient_df)
    reqs = patient_df["request_id"].unique()
    total_time = 0
    for request in reqs:
        req_df = patient_df[patient_df["request_id"] == request]
        # display(req_df)
        start_time = req_df.iloc[0].time
        end_time = req_df[req_df["nurse_phase"] == "arrived"].iloc[0]["time"] #surely this can be done more elegantly
        # end_time = req_df.iloc[-3]["time"] would work here but not in the future
        # print(end_time)
        time_waiting = end_time - start_time
        # print(start_time, end_time, time_waiting)
        total_time += time_waiting
    return total_time

print(total_wait_time(1))
# reqs

,time,type,request_id,patient_id,requested_nurse_id,chosen_nurse_id,pushed_back,nurse_id,nurse_phase,pushed_back_time
6,70.142723,request,3,1,1.0,NaN,True,NaN,NaN,96.356606
8,72.419672,request,5,1,1.0,NaN,True,NaN,NaN,96.356606
13,96.356606,request,3,1,1.0,NaN,True,NaN,NaN,96.356606
15,96.356606,request,5,1,1.0,NaN,True,NaN,NaN,96.356606
18,96.356606,request,3,1,1.0,NaN,True,NaN,NaN,103.946804
...,...,...,...,...,...,...,...,...,...,...
1688,3592.213059,request,164,1,1.0,0.0,False,NaN,NaN,NaN
1691,3593.213059,nurse_phase,164,1,NaN,NaN,NaN,0.0,departing,NaN
1693,3603.213059,nurse_phase,164,1,NaN,NaN,NaN,0.0,arrived,NaN
1701,3623.213059,nurse_phase,164,1,NaN,NaN,NaN,0.0,dealt with request,NaN


4056.3177671239787


In [142]:
len(df[df["patient_id"] == 1]["request_id"].unique())

64